In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [30]:
design.get_chip_z('Q_chip')

'5um'

In [3]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='20um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.8 
    resonator_length_2.append(a)
print(resonator_length_2)


[7.653254683870284, 7.424788339841421, 7.2083465402091305]


# Flipchip

In [6]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-150}um",
                                              pos_y=f"{q0y+140}um",
                                              pad_width="150um",
                                              pad_height="30um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [39]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [40]:
hfss.start()

INFO 12:11PM [connect_project]: Connecting to Ansys Desktop API...
INFO 12:11PM [load_ansys_project]: 	Opened Ansys App
INFO 12:11PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 12:11PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project28
INFO 12:11PM [connect_design]: No active design found (or error getting active design).
INFO 12:11PM [connect]: 	 Connected to project "Project28". No design detected


True

In [41]:
hfss.activate_ansys_design("flip20_res7.4mm", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

12:11PM 47s WARNING [activate_ansys_design]: The design_name=flip20_res7.4mm was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 12:11PM [connect_design]: 	Opened active design
	Design:    flip20_res7.4mm [Solution type: Eigenmode]
WARNING 12:11PM [connect_setup]: 	No design setup detected.
WARNING 12:11PM [connect_setup]: 	Creating eigenmode default setup.
INFO 12:11PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [28]:
hfss.close()

True

# LOM Analysis

In [12]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [13]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.55',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.55',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [15]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 12:41PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmp9retct_4.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5550000000, Maxwell, 1, False
INFO 12:41PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpv11nkb1i.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5550000000, Maxwell, 1, False
INFO 12:41PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpkk0r8yqz.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5550000000, Maxwell, 2, False
INFO 12:41PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpxxzy2o6h.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5550000000, Maxwell, 3, False
INFO 12:41PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpfrz2s7re.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5550000000, Maxwell, 4, False
INFO 12:41

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,276.230,-73.083000,-71.4210,-65.656000
pad_bot_Q0,-73.083,106.110000,-32.1580,-0.087639
pad_top_Q0,-71.421,-32.158000,106.0600,-1.079800
quad_coupler_c0,-65.656,-0.087639,-1.0798,67.002000


In [16]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.94
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.422446 [GHz]
EC 274.462678 [MHz]
EJ 14.854145 [GHz]
alpha -313.118144 [MHz]
dispersion 5.352569 [KHz]
Lq 10.995564 [nH]
Cq 70.575090 [fF]
T1 13371.230523 [us]

**Coupling Properties**

tCqbus1 -0.498765 [fF]
gbus1_in_MHz -7.164203 [MHz]
χ_bus1 -0.011785 [MHz]
1/T1bus1 11.902790 [Hz]
T1bus1 13371.230523 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.736681,309.363902,14.854145,-356.604176,17.209135,[-39.589901733370944],[-0.6026840542382478],0.602684,39.589902
2,5.608739,294.867816,14.854145,-338.447166,10.869888,[-5.5445134909433325],[-0.009497281371129429],0.009497,5.544513
3,5.568706,290.412972,14.854145,-332.89451,9.372928,[-5.746300759313935],[-0.009551653868770844],0.009552,5.746301
4,5.519706,285.012448,14.854145,-326.18025,7.795052,[-6.312080606168811],[-0.010650255742378567],0.010650,6.312081
5,5.463666,278.906359,14.854145,-318.611349,6.287437,[-6.967921696855674],[-0.011881530504473454],0.011882,6.967922
6,5.4484,277.25594,14.854145,-316.569652,5.925258,[-7.082072656607579],[-0.011986464448246024],0.011986,7.082073
7,5.422446,274.462678,14.854145,-313.118144,5.352569,[-7.1642029462603665],[-0.011785406141872342],0.011785,7.164203


In [17]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
